In [1]:
import numpy as np
from proofreader.utils.io import read_cremi_volume, from_h5
%load_ext autoreload
%autoreload 2

In [8]:
import matplotlib.pyplot as plt
import IPython

def view_volume(volume, fig_size=6.5):
    length = volume.shape[0]

    # set up
    fig = plt.figure()
    fig.set_size_inches(fig_size, fig_size)
    axes = fig.add_subplot()
    hfig = display(fig, display_id=True)

    # display
    for i in range(length):
        axes.imshow(volume[i], cmap='gray', interpolation='none',
                    filternorm=False, resample=False)
        fig.canvas.draw()
        hfig.update(fig)
        if i != length-1:
            plt.cla()
    # clean up
    IPython.display.clear_output()


In [2]:
trueA = read_cremi_volume('A',seg=True, path='../../dataset/cremi')
trueB = read_cremi_volume('B',seg=True, path='../../dataset/cremi')
trueC = read_cremi_volume('C',seg=True, path='../../dataset/cremi')

In [3]:
imgA = read_cremi_volume('A', img=True, path='../../dataset/cremi')
imgB = read_cremi_volume('B', img=True, path='../../dataset/cremi')
imgC = read_cremi_volume('C', img=True, path='../../dataset/cremi')

In [33]:
def get_classes_sorted_by_volume(vol, reverse=False, return_counts=False):

    classes, counts = np.unique(vol, return_counts=True)

    sort_indices = np.argsort(counts)
    if reverse:
        sort_indices = np.flip(sort_indices)
    classes = classes[sort_indices]
    if return_counts:
        counts = counts[sort_indices]
        return classes, counts
    return classes

In [34]:
import random
import numpy as np

def sample_square_radius(vol, radius):
    dims = len(vol.shape)
    if type(radius) != tuple:
        radius = (radius,)*dims
    assert dims == len(radius), 'radius must be specified for each dim of input, or be scalar'

    sample_slice = []
    for d in range(dims):
        r = radius[d]
        s = vol.shape[d]
        x = random.randint(r, s-r)
        sample_slice.append(slice(x-r,x+r))
    sample_slice = tuple(sample_slice)
    return vol[sample_slice]


In [39]:
crp = sample_square_radius(trueB, (10,100,100))
clses, cnts = get_classes_sorted_by_volume(crp, return_counts=True)
print(len(clses))


21


33
